# Análise de Perfomance de Estudantes com Machine Learning

Este projeto utiliza um modelo de regressão baseado no algoritmo Random Forest Regressor para prever a nota final dos alunos em um exame, com base em diversos fatores relacionados aos seus hábitos de vida. O Random Forest Regressor é um algoritmo de aprendizado supervisionado que utiliza várias árvores de decisão para realizar previsões numéricas (regressão). Ele é baseado no princípio do ensemble learning, onde múltiplos modelos (neste caso, árvores) são combinados para melhorar a precisão da predição, sendo ideal para para tarefas com múltiplas features.

O conjunto de dados utilizado é sintético e simula registros de 1.000 estudantes, contendo mais de 15 variáveis que influenciam o desempenho acadêmico, como:

- Horas de estudo por dia
- Padrões de sono
- Uso de redes sociais
- Qualidade da alimentação
- Saúde mental
- E, claro, a nota final no exame

Este dataset foi criado com padrões realistas para fins educacionais e é ideal para projetos de aprendizado de máquina (ML), análise de regressão, agrupamento de dados (clustering) e visualização de dados. Já se perguntou quanto tempo no Netflix, no TikTok ou dormindo pode afetar suas notas? Este dataset tenta responder isso simulando os hábitos diários de estudantes e relacionando-os com o desempenho acadêmico.

O dataset pode ser encontrado no seguinte link https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance

In [2]:
# Importação das Bibliotecas Necessárias:
import joblib
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [3]:
# Carregando o dataset com os dados do arquivo csv:
students_dataset = pd.read_csv('../data/student_habits_performance.csv', encoding='utf-8')

In [4]:
# Visualizando os primeiros dados do dataset:
students_dataset.head(10)

,student_id,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
0,S1000,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2
1,S1001,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0
2,S1002,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3
3,S1003,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8
4,S1004,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4
5,S1005,24,Male,7.2,1.3,0.0,No,82.9,7.4,Fair,1,Master,Average,4,No,100.0
6,S1006,21,Female,5.6,1.5,1.4,Yes,85.8,6.5,Good,2,Master,Poor,4,No,89.8
7,S1007,21,Female,4.3,1.0,2.0,Yes,77.7,4.6,Fair,0,Bachelor,Average,8,No,72.6
8,S1008,23,Female,4.4,2.2,1.7,No,100.0,7.1,Good,3,Bachelor,Good,1,No,78.9
9,S1009,18,Female,4.8,3.1,1.3,No,95.4,7.5,Good,5,Bachelor,Good,10,Yes,100.0


In [5]:
# Removendo as features que não são ideais para o modelo:
students_dataset = students_dataset.drop(['student_id', 'age', 'gender'], axis=1)

In [6]:
# Visualizando se as features foram removidas corretamente:
students_dataset.head(10)

,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
0,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2
1,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0
2,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3
3,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8
4,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4
5,7.2,1.3,0.0,No,82.9,7.4,Fair,1,Master,Average,4,No,100.0
6,5.6,1.5,1.4,Yes,85.8,6.5,Good,2,Master,Poor,4,No,89.8
7,4.3,1.0,2.0,Yes,77.7,4.6,Fair,0,Bachelor,Average,8,No,72.6
8,4.4,2.2,1.7,No,100.0,7.1,Good,3,Bachelor,Good,1,No,78.9
9,4.8,3.1,1.3,No,95.4,7.5,Good,5,Bachelor,Good,10,Yes,100.0


In [7]:
# Mapeando os dados categóricos para valores numéricos.
students_dataset['part_time_job'] = students_dataset['part_time_job'].replace(
    {'No': 0,
     'Yes': 1
     }).infer_objects()

students_dataset['diet_quality'] = students_dataset['diet_quality'].replace(
    {'Fair': 0,
     'Good': 1,
     'Poor': 2
     }).infer_objects()

students_dataset['parental_education_level'] = students_dataset['parental_education_level'].replace(
    {'None': 0,
     'High School': 1,
     'Bachelor': 2,
     'Master': 3
     }).infer_objects()

students_dataset['internet_quality'] = students_dataset['internet_quality'].replace(
    {'Good': 0,
     'Average': 1,
     'Poor': 2
     }).infer_objects()

students_dataset['extracurricular_participation'] = students_dataset['extracurricular_participation'].replace(
    {'No': 0,
     'Yes': 1
     }).infer_objects()

/tmp/ipykernel_14668/3988720837.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  students_dataset['part_time_job'] = students_dataset['part_time_job'].replace(
/tmp/ipykernel_14668/3988720837.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  students_dataset['diet_quality'] = students_dataset['diet_quality'].replace(
/tmp/ipykernel_14668/3988720837.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(cop

In [8]:
# Visualizando se os dados foram mapeados corretamente:
students_dataset.head()

,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
0,0.0,1.2,1.1,0,85.0,8.0,0,6,3.0,1,8,1,56.2
1,6.9,2.8,2.3,0,97.3,4.6,1,6,1.0,1,8,0,100.0
2,1.4,3.1,1.3,0,94.8,8.0,2,1,1.0,2,1,0,34.3
3,1.0,3.9,1.0,0,71.0,9.2,2,4,3.0,0,1,1,26.8
4,5.0,4.4,0.5,0,90.9,4.9,0,3,3.0,0,1,0,66.4


In [9]:
# Separando as features e a classe para o modelo:
features, classe = students_dataset.drop(['exam_score'], axis=1), students_dataset['exam_score']

In [10]:
# Separando os conjuntos de dados para treinamento e validação:
features_treinamento, features_validacao, classe_treinamento, classe_validacao = train_test_split(features, classe, test_size=0.30)

In [11]:
# Treinando o modelo de RandomForest:
modelo_RFR = RandomForestRegressor()
modelo_RFR.fit(features_treinamento, classe_treinamento)

RandomForestRegressor()

In [12]:
# Fazendo a predição usando os dados de validação:
classe_predicao = modelo_RFR.predict(features_validacao)

# Métricas de Avaliação
Para avaliar o desempenho do modelo de regressão, foram utilizadas três métricas principais:

A Média Quadrática de Erro (MSE) calcula o erro médio ao quadrado entre os valores reais e os previstos. Por penalizar erros maiores mais severamente, ela é útil para identificar grandes discrepâncias entre o modelo e os dados.

O R² Score (ou Coeficiente de Determinação) mede o quanto o modelo consegue explicar a variância dos dados. Um valor próximo de 1 indica que o modelo faz boas previsões, enquanto valores baixos indicam que ele pouco explica a variabilidade dos resultados.

A Média Absoluta de Erro (MAE) representa a média das diferenças absolutas entre os valores reais e os previstos. Por ser uma métrica mais direta e fácil de interpretar, ela indica, em média, o quanto o modelo erra para mais ou para menos.

In [13]:
# Métricas:
print(f"Média Quadrática de Erro: {mean_squared_error(classe_validacao, classe_predicao):.2f}")
print(f"R² Score: {r2_score(classe_validacao, classe_predicao):.2f}")
print(f"Média Absoluta de Erro: {mean_absolute_error(classe_validacao, classe_predicao):.2f}")

Média Quadrática de Erro: 41.28
R² Score: 0.84
Média Absoluta de Erro: 5.16


# Gerando o Modelo para a Aplicação:

In [16]:
joblib.dump(modelo_RFR, "../src/model/students_habits_perfomance.pkl")
print("modelo salvo!")

modelo salvo!
